In [1]:
import struct
import datetime as dt
import glob
import os
from collections import namedtuple
import matplotlib.image as mpimg
import cv2

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from sklearn.metrics import average_precision_score

# true_labels: true labels (1 for positive, 0 for negative)
# predicted_scores: confidence scores predicted by the model
true_path= os.path.join('drive', 'My Drive', '3DRP project/runs/gt_image_2')
true=[f for f in os.listdir(true_path) if f.endswith(".png")]
true.sort()

predicted_path= os.path.join('drive', 'My Drive', '3DRP project/runs/30epoch_only_rgb_road')
predicted=[f for f in os.listdir(predicted_path) if f.endswith(".png")]
predicted.sort()

true_labels=[]
predicted_scores=[]


for lidar_file, image_file in zip(true, predicted):
    # Construct the full paths
    tru_pth= os.path.join(true_path, lidar_file)
    pred_pth = os.path.join(predicted_path, image_file)
    # file_path_calib = os.path.join(calib_path, calib_file)

    tru1= cv2.imread(tru_pth)
    pred1= cv2.imread(pred_pth)

    # tru1.flatten()
    # pred1.flatten()
    tru1= tru1[:,:,0:]
    pred1= tru1[:,:,0:]


    true_labels.append(tru1)
    predicted_scores.append(pred1)
    average_precision = average_precision_score(tru1, pred1)
    print(f'Average Precision: {average_precision}')

    # exit()

true_labels=np.array(true_labels)
predicted_scores=np.array(predicted_scores)


# average_precision = average_precision_score(true_labels, predicted_scores)
# average_precision = average_precision_score(tru1, pred1)


print(f'Average Precision: {average_precision}')


ValueError: ignored

In [ ]:
    def get_velo_scans(bin_path):
        obj = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
        return obj

In [ ]:
"""Read in a calibration file and parse into a dictionary."""
def read_calib_file(filepath):
    data = {}

    with open(filepath, 'r') as f:
        for line in f.readlines():
            key, value = line.split(':', 1)
            # The only non-float values in these files are dates, which
            # we don't care about anyway
            try:
                data[key] = np.array([float(x) for x in value.split()])
            except ValueError:
                pass

    return data

"""
  Filter camera angles for KiTTI Datasets
"""
def filter_by_camera_angle(pc):
    bool_in = np.logical_and((pc[:, 1] < pc[:, 0] - 0.27), (-pc[:, 1] < pc[:, 0] - 0.27))
    """
    /*
      * @brief KiTTI Velodyne Coordinate
      *          |x(forward)
      *      C   |   D
      *          |
      *  y---------------
      *          |
      *      B   |   A
      */
    """
    # bool_in = np.where(pc[:, 0] > 0)
    return pc[bool_in]

In [ ]:
from matplotlib import cm

class Color(object):
    jet = None

    """
      https://github.com/ros-visualization/rviz/blob/kinetic-devel/src/rviz/default_plugin/point_cloud_transformers.cpp
      :return
        Rainbow color (rgb8) from val in [0., 1.]
    """
    @staticmethod
    def get_rainbow_color(val, min_val=0., diff=255):
        value = 1.0 - (val - min_val)
        # restrict value between 0 and 1
        value = max(value, 0.)
        value = min(value, 1.)

        h = value * 5.0 + 1.0
        i = int(h)
        f = h - i
        # if i is even
        if not i&1:
            f = 1 - f
        n = int((1 - f)*diff)
        bgr = [0]*3
        if i <= 1:
            bgr[2] = n; bgr[1] = 0; bgr[0] = 255
        elif i == 2:
            bgr[2] = 0; bgr[1] = n; bgr[0] = 255
        elif i == 3:
            bgr[2] = 0; bgr[1] = 255; bgr[0] = n
        elif i == 4:
            bgr[2] = n; bgr[1] = 255; bgr[0] = 0
        elif i >= 5:
            bgr[2] = 255; bgr[1] = n; bgr[0] = 0
        # print bgr
        return bgr


    def get_heat_color(point):

        rgb = [0]*3

        if point[2] <= -1.67:
            rgb[2] = 255; rgb[1] = 150-(point[2]/-2)*150; rgb[0] = 0

        elif  point[2] <= -1.63 and point[2] > -1.67:
            rgb[2] = 255; rgb[1] = 125; rgb[0] = 0
        elif  point[2] <= -1.6 and point[2] > -1.63:
            rgb[2] = 255; rgb[1] = 200; rgb[0] = 0
        elif  point[2] <= -1.55 and point[2] > -1.6:
            rgb[2] = 255; rgb[1] = 255; rgb[0] = 0
        elif  point[2] <= -1.4 and point[2] > -1.55:
            rgb[2] = 0; rgb[1] = 255; rgb[0] = 0
        elif  point[2] <= -1.2 and point[2] > -1.4:
            rgb[2] = 0; rgb[1] = 255; rgb[0] = 125
        elif  point[2] <= 0 and point[2] > -1.2:
            rgb[2] = 0; rgb[1] = 75; rgb[0] = 255
        else:
            rgb[2] = 0; rgb[1] = 0; rgb[0] = 255

        return rgb


In [ ]:
"""
  1. Color Point Cloud from RGB Image
  2. Project Point Cloud into RGB Image for depth
  3. Also apply to Gray Image
"""
def lidar_camera_fusion(point_cloud, image, T_velo_cam, P_velo_image):
    image_size = image.shape

    is_gray = len(image_size) < 3
    if is_gray:
        print("LiDAR Gray-Image fusing...")
    else:
        print("LiDAR RGB-Image fusing...")

    image_depth = image.copy()

    # XYZRGB point cloud
    #modify these for old
    pc_rgb = np.zeros((point_cloud.shape[0], 4), dtype=np.float32)
    pc_rgb_6 = np.zeros((point_cloud.shape[0], 6), dtype=np.float32)

    pc_rgb[:, :3] = point_cloud[:, :3]
    pc_rgb_6[:, :3] = point_cloud[:, :3]


    xyz = point_cloud.copy()
    xyz[:,3] = 1.0
    # project into image
    velo_img = np.dot(P_velo_image, xyz.T).T
    # normalize homogeneous coordinates
    velo_img = np.true_divide(velo_img[:,:2], velo_img[:,[-1]])
    velo_img = np.round(velo_img).astype(np.uint16)

    # compute depth in Camera coordinate
#         pc_img = np.dot(T_velo_cam, xyz.T).T
#         depth = np.sqrt(np.square(pc_img[:, 0]) + np.square(pc_img[:, 1]) + np.square(pc_img[:, 2]))
#         depth_min = min(depth)
#         depth_max = max(depth)
#         depth = depth / (depth_max - depth_min)

    height = point_cloud[:,2]
    height_min = min(height)
    height_max = min(height)
    height = height / height_max


    if is_gray:
        for pt in range(0, velo_img.shape[0]):
            row_idx = velo_img[pt][1]
            col_idx = velo_img[pt][0]

            if (row_idx >= 0 and row_idx < image_size[0]) \
                    and (col_idx >= 0 and col_idx < image_size[1]):
                # assign image color to point cloud
                color =   (image[row_idx][col_idx] << 16) \
                        | (image[row_idx][col_idx] << 8) \
                        | image[row_idx][col_idx]
                pc_rgb[pt, 3] = color

                # assign point cloud to image pixel
                cv2.circle(image_depth, (col_idx,row_idx), 1, depth[pt] * 255, thickness=-1)
    else:
        for pt in range(0, velo_img.shape[0]):
            row_idx = velo_img[pt][1]
            col_idx = velo_img[pt][0]

            if (row_idx >= 0 and row_idx < image_size[0]) \
                    and (col_idx >= 0 and col_idx < image_size[1]):
                # assign image color to point cloud
#                     color =   (image[row_idx][col_idx][2] << 16) \
#                               | (image[row_idx][col_idx][1] << 8) \
#                               | image[row_idx][col_idx][0]
                #modify these for old values
                color = image[row_idx][col_idx][2]
                pc_rgb[pt, 3] = color
                colors = image[row_idx][col_idx][:]
                pc_rgb_6[pt, 3:] = colors

                # assign point cloud to image pixel
                # use rainbow color band
                cv_color = Color.get_heat_color(point_cloud[pt])
#                     cv_color = Color.get_rainbow_color(height[pt])
                # image_depth[row_idx][col_idx] = cv_color
                cv2.circle(image_depth, (col_idx,row_idx), 1, cv_color, thickness=-1)

    return pc_rgb, image_depth, pc_rgb_6, velo_img

In [ ]:
import re

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "3DRP project/3d perp/Project/data_toad_velodyne/training/velodyne/"
Lidar_path = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)


# Your text data
GOOGLE_DRIVE_PATH_AF = "3DRP project/3d perp/Project/data_road/training/calib"
calib_path= os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AF)


GOOGLE_DRIVE_PATH_AFTER = "3DRP project/3d perp/Project/data_road/training/image_2"
cam_path= os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER)


GOOGLE_DRIVE_PATH_1 = "3DRP project/csvs/pc_rgb"
GOOGLE_DRIVE_PATH_2 = "3DRP project/csvs/pc_rgb_6"
GOOGLE_DRIVE_PATH_3 = "3DRP project/Project/fused_data/npy"
GOOGLE_DRIVE_PATH_0 = "3DRP project/Project/fused_data/png"




# List all files in each directory
lidar_files = [f for f in os.listdir(Lidar_path) if f.endswith(".bin")]
image_files = [f for f in os.listdir(cam_path) if f.endswith(".png")]
calib_files= [f for f in os.listdir(calib_path) if f.endswith(".txt")]

# Sort the files to ensure that files with similar names correspond to each other
lidar_files.sort()
image_files.sort()
calib_files.sort()



# Iterate over the files with similar names
for lidar_file, image_file, calib_file in zip(lidar_files, image_files, calib_files):
    # Construct the full paths
    filename_pc= os.path.join(Lidar_path, lidar_file)
    img_path_png = os.path.join(cam_path, image_file)
    file_path_calib = os.path.join(calib_path, calib_file)


    pc = get_velo_scans(filename_pc)
    # print(pc.shape)
    pc = filter_by_camera_angle(pc)


        # Open the file in read mode ('r')
    with open(file_path_calib, 'r') as file:
        # Read the contents of the file
        file_contents = file.read()

    # Now, file_contents contains the contents of the text file
    # print(file_contents)

    text_data= file_contents


    # Extract P1 data using regular expression
    pattern = re.compile(r'P2: (.+)')
    match = pattern.search(text_data)

    if match:
        p1_data_str = match.group(1)
        p2_data = np.fromstring(p1_data_str, sep=' ')
        # print("P1 Data as NumPy array:", p2_data_np)
    else:
        print("P1 Data not found.")


    # Extract Rect data using regular expression
    pattern = re.compile(r'R0_rect: (.+)')
    match = pattern.search(text_data)

    if match:
        p1_data_str = match.group(1)
        rect_data = np.fromstring(p1_data_str, sep=' ')
        # print("Rect Data as NumPy array:", p1_data_np)
    else:
        print("Rect Data not found.")


    # Extract Tr_velo_to_cam data using regular expression
    pattern = re.compile(r'Tr_velo_to_cam: (.+)')
    match = pattern.search(text_data)

    if match:
        p1_data_str = match.group(1)
        velo_cam_data= np.fromstring(p1_data_str, sep=' ')
        # print("velo_cam Data as NumPy array:", p1_data_np)
    else:
        print("velo_cam Data not found.")


    # To project a 3D point x in reference camera coordinates to a point y on the i'th image plane,
    # the rectifying rotation matrix of the reference camera: R_rect_00 must be considered as well.
    R_rect_00 = np.zeros((4, 4))

    R_rect_00[:3,:3] = rect_data.reshape(-1, 3)


    R_rect_00[3,3] = 1.  #instead can divide by the end term

    P_rect_02 = np.zeros((3, 4))

    P_rect_02 = p2_data.reshape(-1, 4)


    vel2cam0 = np.zeros((4, 4))

    vel2cam0[:3,:4] = velo_cam_data.reshape(-1, 4)


    vel2cam0[3,3] = 1.   #instead can divide by the last term
    vel2cam0 = vel2cam0/vel2cam0[3,3]   #instead can divide by the last term


    T_velo_to_cam = np.dot(R_rect_00, vel2cam0)

    P_velo_to_img = np.dot(P_rect_02, np.dot(R_rect_00, vel2cam0))

    image = cv2.imread(img_path_png)

    # XYZRGB point cloud and depth image
    pc_rgb, image_depth, pc_rgb_6, velo_image = lidar_camera_fusion(pc, image, T_velo_to_cam, P_velo_to_img)

    # print(pc_rgb[300, :])
    out_path= os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_0,image_file)
    cv2.imwrite(out_path, pc_rgb)
    # cv2.imwrite(out_path, image_depth)


    name= calib_file.replace('txt','npy')
    out_path_rgb= os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_1,name)
    out_path_rgb_6= os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_2,name)
    out_path_fused= os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_3,name)


    # Example headers
    header_rgb = ['X', 'Y', 'Z', 'Color']
    header_rgb_6 = ['X', 'Y', 'Z', 'R', 'G', 'B']


    # Save the NumPy array to a CSV file with headers
    # np.savetxt(out_path_rgb, pc_rgb, delimiter=',', header=','.join(header_rgb), comments='')
    # np.savetxt(out_path_rgb_6, pc_rgb_6, delimiter=',', header=','.join(header_rgb_6), comments='')

    pcd=pc[:,:3]
    data= np.hstack((velo_image,pcd))
    y_lim, x_lim,_=image.shape
    #filter out non valid pixles
    data = data[(data[:, 0] >= 0) & (data[:, 0] < x_lim) & (data[:, 1] >= 0) & (data[:, 1] < y_lim)]

    final_data=np.zeros(((y_lim,x_lim,6)))
    final_data[:,:,3:6]=image
    coords= data[:,0:2]  #[x,y]
    coords=coords.astype(int)
    xyz= data[:,2:5]

    final_data[coords[:,1],coords[:,0],0:3]= xyz
    np.save(out_path_fused, final_data)
    # np.savetxt(out_path_fused, finalnow how do i train these models with input as _data, delimiter=',', header=','.join(header_rgb_6), comments='')









LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Image fusing...
LiDAR RGB-Im